In [2]:
import os
from pathlib import Path
import shutil

# Prefix for the folders that will store grouped files
TYPE_DIR_PREFIX = "__type__"  # e.g. __type__pdf, __type__xlsx, __type__noext


def scan_file_types(root: Path):
    """
    Walk through 'root' and collect:
      - ext_counts: {ext_key: count}
      - files_by_ext: {ext_key: [Path, Path, ...]}
    ext_key is like 'pdf', 'txt', or 'noext' for files with no extension.
    """
    ext_counts = {}
    files_by_ext = {}

    # Walk top-down so we can modify `dirnames` to skip our own type folders
    for dirpath, dirnames, filenames in os.walk(root, topdown=True):
        # Skip the folders we create ourselves to avoid re-processing them
        dirnames[:] = [d for d in dirnames if not d.startswith(TYPE_DIR_PREFIX)]

        for name in filenames:
            file_path = Path(dirpath) / name

            # Get extension (last part, e.g. ".pdf"), normalised to lowercase
            ext = file_path.suffix.lower()
            if ext:
                ext_key = ext.lstrip(".")  # ".pdf" -> "pdf"
            else:
                ext_key = "noext"

            ext_counts[ext_key] = ext_counts.get(ext_key, 0) + 1
            files_by_ext.setdefault(ext_key, []).append(file_path)

    return ext_counts, files_by_ext


def print_summary(root: Path, ext_counts: dict, files_by_ext: dict, max_examples: int = 5):
    """
    Print a human-friendly summary of what was found.
    """
    total_files = sum(ext_counts.values())
    print("\n=== Scan summary ===")
    print(f"Root folder: {root}")
    print(f"Total files found (excluding {TYPE_DIR_PREFIX} folders): {total_files}\n")

    if not ext_counts:
        print("No files found.")
        return

    print("File types detected (sorted by count):")
    for ext_key, count in sorted(ext_counts.items(), key=lambda x: (-x[1], x[0])):
        if ext_key == "noext":
            label = "(no extension)"
        else:
            label = f".{ext_key}"
        print(f"  {label:15} -> {count} file(s)")

    # Show a few example files for each type
    print("\nExample files per type (up to "
          f"{max_examples} each):")
    for ext_key, files in sorted(files_by_ext.items(), key=lambda x: (-len(x[1]), x[0])):
        if ext_key == "noext":
            label = "(no extension)"
        else:
            label = f".{ext_key}"

        print(f"\n{label} - {len(files)} file(s):")
        for f in files[:max_examples]:
            try:
                rel = f.relative_to(root)
            except ValueError:
                rel = f
            print(f"   - {rel}")


def safe_move_file(src: Path, dest_folder: Path):
    """
    Move 'src' into 'dest_folder'. If a file with the same name
    already exists, append __1, __2, ... before the extension.
    """
    dest_folder.mkdir(parents=True, exist_ok=True)

    target = dest_folder / src.name
    if not target.exists():
        shutil.move(str(src), str(target))
        return

    stem = src.stem
    suffix = src.suffix
    counter = 1
    while True:
        new_name = f"{stem}__{counter}{suffix}"
        candidate = dest_folder / new_name
        if not candidate.exists():
            shutil.move(str(src), str(candidate))
            break
        counter += 1


def group_files_by_type(root: Path, files_by_ext: dict):
    """
    Actually move files into __type__<ext> folders inside 'root'.
    """
    for ext_key, file_list in files_by_ext.items():
        # Folder name for this type
        if ext_key == "noext":
            folder_name = f"{TYPE_DIR_PREFIX}_noext"
        else:
            folder_name = f"{TYPE_DIR_PREFIX}_{ext_key}"

        dest_dir = root / folder_name

        print(f"\nMoving {len(file_list)} file(s) to: {dest_dir}")
        for src in file_list:
            safe_move_file(src, dest_dir)


def main():
    # Your root folder (hard-coded)
    root = Path(r"C:\Users\Julian.Diaz\OneDrive - XENITH CONSULTING PTY LTD\Documents\04_eXgen\B").expanduser().resolve()

    if not root.exists():
        print(f"Path does not exist: {root}")
        return
    if not root.is_dir():
        print(f"Path is not a directory: {root}")
        return

    # 1) Scan / initial query
    ext_counts, files_by_ext = scan_file_types(root)
    print_summary(root, ext_counts, files_by_ext)

    if not ext_counts:
        return

    # 2) Confirm before moving anything
    print("\nDo you want to group these files by type into new folders inside this root?")
    print(f"(Existing files will be moved into subfolders starting with '{TYPE_DIR_PREFIX}_')")
    confirm = input("Type 'yes' to proceed, anything else to cancel: ").strip().lower()

    if confirm != "yes":
        print("Cancelled. No files were moved.")
        return

    # 3) Perform the grouping
    group_files_by_type(root, files_by_ext)
    print("\nDone! Files have been grouped by type.")


if __name__ == "__main__":
    main()



=== Scan summary ===
Root folder: C:\Users\Julian.Diaz\OneDrive - XENITH CONSULTING PTY LTD\Documents\04_eXgen\B
Total files found (excluding __type__ folders): 2927

File types detected (sorted by count):
  .tif            -> 1674 file(s)
  .jpeg           -> 306 file(s)
  .json           -> 256 file(s)
  .pdf            -> 242 file(s)
  .docx           -> 209 file(s)
  .zip            -> 189 file(s)
  .txt            -> 46 file(s)
  .xlsx           -> 5 file(s)

Example files per type (up to 5 each):

.tif - 1674 file(s):
   - 0081b400-ec7f-4091-8a04-f43a463f17d6\WHOLE_REPORT_FOR_ORGANIC_ANALYSIS_OF_35_CORE_PLUG_SAMPLES_FROM_THE_SURAT_BOWEN_DENISON_AND_TAROOM_TROUGHS_AND_CARPENTARIA_BASINS_EXTRACT_FROM_REPORT_PART_1..tif
   - 00aec92b-a596-4456-bd5b-a5ce13092877\APPENDIX_3__ROCK_CHIP_SAMPLE_LEDGER.tif
   - 00aec92b-a596-4456-bd5b-a5ce13092877\APPENDIX_5__SOIL_SAMPLE_LEDGERS.tif
   - 00aec92b-a596-4456-bd5b-a5ce13092877\APPENDIX_7__PETROLOGY_REPORTS.tif
   - 00aec92b-a596-4456-bd5b-a

Type 'yes' to proceed, anything else to cancel:  yes



Moving 209 file(s) to: C:\Users\Julian.Diaz\OneDrive - XENITH CONSULTING PTY LTD\Documents\04_eXgen\B\__type___docx

Moving 256 file(s) to: C:\Users\Julian.Diaz\OneDrive - XENITH CONSULTING PTY LTD\Documents\04_eXgen\B\__type___json

Moving 1674 file(s) to: C:\Users\Julian.Diaz\OneDrive - XENITH CONSULTING PTY LTD\Documents\04_eXgen\B\__type___tif

Moving 189 file(s) to: C:\Users\Julian.Diaz\OneDrive - XENITH CONSULTING PTY LTD\Documents\04_eXgen\B\__type___zip

Moving 306 file(s) to: C:\Users\Julian.Diaz\OneDrive - XENITH CONSULTING PTY LTD\Documents\04_eXgen\B\__type___jpeg

Moving 242 file(s) to: C:\Users\Julian.Diaz\OneDrive - XENITH CONSULTING PTY LTD\Documents\04_eXgen\B\__type___pdf

Moving 46 file(s) to: C:\Users\Julian.Diaz\OneDrive - XENITH CONSULTING PTY LTD\Documents\04_eXgen\B\__type___txt

Moving 5 file(s) to: C:\Users\Julian.Diaz\OneDrive - XENITH CONSULTING PTY LTD\Documents\04_eXgen\B\__type___xlsx

Done! Files have been grouped by type.


In [4]:
import os
from PIL import Image
import fitz  # PyMuPDF
import pytesseract

# Point pytesseract to your Tesseract installation
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Folder with your .tif "PDF" files (source)
BASE_FOLDER = r"C:\Users\Julian.Diaz\OneDrive - XENITH CONSULTING PTY LTD\Documents\04_eXgen\B\__type___tif"

# Folder where OCR .txt and merged files will be written (destination)
OUTPUT_FOLDER = r"C:\Users\Julian.Diaz\OneDrive - XENITH CONSULTING PTY LTD\Documents\04_eXgen\B\__type___tif_OCR"

# Create output folder if it does not exist
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Max size per merged file: 200 MB
MAX_BYTES = 200 * 1024 * 1024  # 200 MB in bytes


def ocr_pdf_like_tif_to_text(pdf_path, txt_path=None, lang="eng"):
    """
    Treats pdf_path as a PDF (even if it ends in .tif):
    renders pages to images, OCRs with Tesseract, saves text to a .txt file
    in OUTPUT_FOLDER.
    """
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]

    if txt_path is None:
        txt_path = os.path.join(OUTPUT_FOLDER, base_name + "_ocr.txt")

    # If we've already OCR'd this file, skip it
    if os.path.exists(txt_path):
        print(f"  -> OCR text already exists, skipping: {os.path.basename(txt_path)}")
        return txt_path

    # Open file as PDF via bytes, ignoring extension
    with open(pdf_path, "rb") as f:
        pdf_bytes = f.read()
    doc = fitz.open(stream=pdf_bytes, filetype="pdf")

    all_text = []

    for page_number in range(len(doc)):
        page = doc[page_number]
        print(f"    [OCR] Page {page_number + 1} of {len(doc)}")

        # Render ~300 dpi for better OCR quality
        zoom = 300 / 72
        mat = fitz.Matrix(zoom, zoom)
        pix = page.get_pixmap(matrix=mat)

        mode = "RGBA" if pix.alpha else "RGB"
        img = Image.frombytes(mode, (pix.width, pix.height), pix.samples)

        gray = img.convert("L")
        text = pytesseract.image_to_string(gray, lang=lang)
        all_text.append(f"===== PAGE {page_number + 1} =====\n{text}\n")

    doc.close()

    with open(txt_path, "w", encoding="utf-8") as f:
        f.write("\n".join(all_text))

    print(f"  -> Text saved to: {txt_path}")
    return txt_path


# ============================
# 1) OCR all .tif files
# ============================

tif_files = [f for f in os.listdir(BASE_FOLDER)
             if f.lower().endswith((".tif", ".tiff"))]
tif_files.sort()

print(f"Found {len(tif_files)} .tif files to OCR in:\n  {BASE_FOLDER}\n")

ocr_txt_paths = []

for i, fname in enumerate(tif_files, start=1):
    full_path = os.path.join(BASE_FOLDER, fname)
    print(f"[{i}/{len(tif_files)}] Processing: {fname}")
    try:
        txt_path = ocr_pdf_like_tif_to_text(full_path, lang="eng")
        ocr_txt_paths.append(txt_path)
    except Exception as e:
        print(f"  !! Error OCR'ing {fname}: {e}")

print("\nOCR step complete.\n")


# ============================
# 2) Merge *_ocr.txt into ≤200 MB chunks
#    (working entirely in OUTPUT_FOLDER)
# ============================

ocr_files = [f for f in os.listdir(OUTPUT_FOLDER)
             if f.lower().endswith("_ocr.txt")]
ocr_files.sort()

print(f"Found {len(ocr_files)} OCR text files to merge in:\n  {OUTPUT_FOLDER}\n")

if not ocr_files:
    print("No *_ocr.txt files found. Check the output folder or naming pattern.")
else:
    merged_files = []

    out_index = 1
    out_path = os.path.join(OUTPUT_FOLDER, f"merged_ocr_part_{out_index:03d}.txt")
    out_file = open(out_path, "w", encoding="utf-8", newline="")
    merged_files.append(out_path)
    current_bytes = 0

    print(f"Starting new merged file: {out_path}")

    for i, fname in enumerate(ocr_files, start=1):
        src_path = os.path.join(OUTPUT_FOLDER, fname)

        file_size = os.path.getsize(src_path)

        # Single file larger than 200 MB: warn
        if file_size > MAX_BYTES:
            print(f"WARNING: {fname} alone is > 200 MB ({file_size} bytes). "
                  "It may exceed NotebookLM's single-file limit.")

        header = f"\n\n===== BEGIN FILE: {fname} =====\n\n"
        header_bytes = len(header.encode("utf-8"))
        total_add_bytes = header_bytes + file_size

        # If adding this file would exceed 200 MB, start a new merged output
        if current_bytes > 0 and current_bytes + total_add_bytes > MAX_BYTES:
            out_file.close()
            out_index += 1
            out_path = os.path.join(OUTPUT_FOLDER, f"merged_ocr_part_{out_index:03d}.txt")
            out_file = open(out_path, "w", encoding="utf-8", newline="")
            merged_files.append(out_path)
            current_bytes = 0
            print(f"\nStarting new merged file: {out_path}")

        print(f"[{i}/{len(ocr_files)}] Adding {fname} to {os.path.basename(out_path)}")

        # Write header
        out_file.write(header)
        current_bytes += header_bytes

        # Stream content
        with open(src_path, "r", encoding="utf-8") as src:
            for line in src:
                encoded = line.encode("utf-8")
                out_file.write(line)
                current_bytes += len(encoded)

    out_file.close()

    print("\nMerging complete. Created the following files:\n")
    for path in merged_files:
        size_bytes = os.path.getsize(path)
        size_mb = size_bytes / (1024 * 1024)
        print(f"  {os.path.basename(path)}  ->  {size_mb:.2f} MB ({size_bytes} bytes)")


Found 1674 .tif files to OCR in:
  C:\Users\Julian.Diaz\OneDrive - XENITH CONSULTING PTY LTD\Documents\04_eXgen\B\__type___tif

[1/1674] Processing: A-P_3904M_CARTRIDGE_CREEK_LOCATION_AND_ACCESS.tif
    [OCR] Page 1 of 1
  -> Text saved to: C:\Users\Julian.Diaz\OneDrive - XENITH CONSULTING PTY LTD\Documents\04_eXgen\B\__type___tif_OCR\A-P_3904M_CARTRIDGE_CREEK_LOCATION_AND_ACCESS_ocr.txt
[2/1674] Processing: A._TO_P._APPLICATIONS_-_MILLUNGERA_AREA.tif
    [OCR] Page 1 of 1
  -> Text saved to: C:\Users\Julian.Diaz\OneDrive - XENITH CONSULTING PTY LTD\Documents\04_eXgen\B\__type___tif_OCR\A._TO_P._APPLICATIONS_-_MILLUNGERA_AREA_ocr.txt
[3/1674] Processing: APPENDICES_1-8.tif
    [OCR] Page 1 of 17
    [OCR] Page 2 of 17
    [OCR] Page 3 of 17
    [OCR] Page 4 of 17
    [OCR] Page 5 of 17
    [OCR] Page 6 of 17
    [OCR] Page 7 of 17
    [OCR] Page 8 of 17
    [OCR] Page 9 of 17
    [OCR] Page 10 of 17
    [OCR] Page 11 of 17
    [OCR] Page 12 of 17
    [OCR] Page 13 of 17
    [OCR] Page 